Upload data from Artists.txt, Artwork.txt, and PainterPalette.csv

In [25]:
import pandas as pd


In [41]:
pp_df = pd.read_csv("c:\\Users\\***REMOVED***\\.vscode\\PainterPalette.csv")  
column_names = [
    "ConstituentID", "DisplayName", "ArtistBio", "Nationality", "Gender",
    "BeginDate", "EndDate", "Wiki QID", "ULAN"
]

# Read the file as CSV, handling missing values
artists_df = pd.read_csv("c:\\Users\\***REMOVED***\\.vscode\\Artists.txt", header=None, names=column_names, dtype={"ConstituentID": str}, na_values=["", " ", "null"])

artworks_df = pd.read_csv("c:\\Users\\***REMOVED***\\.vscode\\Artworks.txt", header=None, names=column_names, dtype={"ConstituentID": str}, na_values=["", " ", "null"])



C:\Users\***REMOVED***\AppData\Local\Temp\ipykernel_27176\1299111328.py:10: DtypeWarning: Columns (17,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  artworks_df = pd.read_csv("c:\\Users\\***REMOVED***\\.vscode\\Artworks.txt", header=None, names=column_names, dtype={"ConstituentID": str}, na_values=["", " ", "null"])


Format for the artists.txt:

  "ConstituentID": 1,  
  "DisplayName": "Robert Arneson",   
  "ArtistBio": "American, 1930–1992",  
  "Nationality": "American",  
  "Gender": "male",  
  "BeginDate": 1930,  
  "EndDate": 1992,  
  "Wiki QID": null,  
  "ULAN": null  



In [42]:
print(artists_df)

       ConstituentID          DisplayName                         ArtistBio  \
0      ConstituentID          DisplayName                         ArtistBio   
1                  1       Robert Arneson               American, 1930–1992   
2                  2       Doroteo Arnaiz                Spanish, born 1936   
3                  3          Bill Arnold               American, born 1941   
4                  4      Charles Arnoldi               American, born 1946   
...              ...                  ...                               ...   
15663         138843      Liliana Maresca            Argentine, 1951 – 1994   
15664         138844          Elba Bairon               Bolivian, born 1947   
15665         138845      Marcia Schvartz                         born 1955   
15666         138858  Anastasia Samoylova  American, born Russia, born 1984   
15667         138892       Sadie Red Wing             Indigenous, born 1990   

       Nationality  Gender  BeginDate  EndDate  Wik

In [43]:
#Check if ConstituentID has duplicates

duplicates = artists_df.duplicated(subset="ConstituentID", keep=False) 
dupe_counts = artists_df["ConstituentID"].value_counts()
dupes = dupe_counts[dupe_counts > 1]  # Filter only duplicates

print(dupes)


Series([], Name: count, dtype: int64)


In [ ]:
#Calculates nulls

import pandas as pd

# Calculate total number of rows
total_rows = len(artists_df)

# Count rows with at least one missing (NaN) value in the specified columns
null_rows = artists_df[["Nationality", "Gender", "BeginDate", "EndDate"]].isnull().any(axis=1).sum()

# Count rows with at least one zero in numerical columns
zero_rows = (artists_df[["BeginDate", "EndDate"]] == 0).any(axis=1).sum()

# Calculate percentages
percent_null_rows = (null_rows / total_rows) * 100
percent_zero_rows = (zero_rows / total_rows) * 100

# Checking for missing values (NaN/null)
missing_counts = artists_df[["Nationality", "Gender", "BeginDate", "EndDate"]].isnull().sum()

# Checking for zero values in numerical columns
zero_counts = (artists_df[["BeginDate", "EndDate"]] == 0).sum()

# Display results
print("Missing (NaN) values per column:")
print(missing_counts)

print("\nZero values per column:")
print(zero_counts)

print(f"\nPercentage of rows with at least one NaN value: {percent_null_rows:.2f}%")
print(f"Percentage of rows with at least one zero value: {percent_zero_rows:.2f}%")


Missing (NaN) values per column:
Nationality    2499
Gender         3276
BeginDate         0
EndDate           0
dtype: int64

Zero values per column:
BeginDate    0
EndDate      0
dtype: int64

Percentage of rows with at least one NaN value: 25.02%
Percentage of rows with at least one zero value: 0.00%


In [ ]:
#Pulls artist dataframes from wikidata

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json
import re

# Define the SPARQL query
query = """
SELECT ?artist ?artistLabel ?nationalityLabel ?birthDate WHERE {
  ?artist wdt:P31 wd:Q5;  # Instance of human
         wdt:P106 wd:Q1028181;  # Occupation: artist
         wdt:P27 ?nationality;  # Country of citizenship
         OPTIONAL { ?artist wdt:P569 ?birthDate. }  # Date of birth (optional to prevent missing values)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 1***REMOVED***  # Limit results to prevent timeouts
"""

# Initialize SPARQL wrapper
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

try:
    # Fetch response safely
    response = sparql.query().response.read().decode("utf-8")

    # Validate if response is JSON
    if not response.strip().startswith("{"):
        raise ValueError("SPARQL endpoint did not return JSON. Response received:\n" + response)

    # Remove invalid control characters that may cause JSONDecodeError
    cleaned_response = re.sub(r"[\x00-\x1F\x7F]", "", response)

    # Convert JSON response safely
    results = json.loads(cleaned_response)

    # Process results into a DataFrame
    data = []
    for result in results.get("results", {}).get("bindings", []):
        data.append({
            "artist": result["artistLabel"]["value"],
            "nationality": result.get("nationalityLabel", {}).get("value", None),
            "birthDate": result.get("birthDate", {}).get("value", None)
        })

    wikidata_df = pd.DataFrame(data)

    # Display the DataFrame
    wiki_len = len(wikidata_df)
    print(f"Total records fetched: {wiki_len}")
    print(wikidata_df.head())

except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
except ValueError as ve:
    print(f"ValueError: {ve}")
except Exception as ex:
    print(f"Unexpected error: {ex}")


Total records fetched: 1***REMOVED***
                  artist     nationality             birthDate
0       Matthias Brandes         Germany  1950-01-01T00:00:00Z
1       Mikhail Larionov          France  1881-06-03T00:00:00Z
2       Mikhail Larionov  Russian Empire  1881-06-03T00:00:00Z
3       André Cluysenaar         Belgium  1872-05-31T00:00:00Z
4  Pierre-Auguste Renoir          France  1841-02-25T00:00:00Z


In [ ]:
#Fetch Records

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import re

# Define the SPARQL query to fetch as many relevant columns as possible
query = """
SELECT ?artist ?artistLabel ?nationalityLabel ?citizenshipLabel ?genderLabel ?birthDate ?deathDate ?birthPlaceLabel ?deathPlaceLabel 
       ?movementLabel ?occupationLabel ?paintingSchoolLabel ?influencedByLabel ?influencedOnLabel ?pupilsLabel ?teachersLabel ?friendsLabel
WHERE {
  ?artist wdt:P31 wd:Q5;  # Instance of human
         wdt:P106 wd:Q1028181.  # Occupation: artist
  OPTIONAL { ?artist wdt:P27 ?nationality. }  # Country of citizenship
  OPTIONAL { ?artist wdt:P27 ?citizenship. }  # Citizenship
  OPTIONAL { ?artist wdt:P21 ?gender. }  # Gender
  OPTIONAL { ?artist wdt:P569 ?birthDate. }  # Date of birth
  OPTIONAL { ?artist wdt:P570 ?deathDate. }  #  Date of death
  OPTIONAL { ?artist wdt:P19 ?birthPlace. }  # Birthplace
  OPTIONAL { ?artist wdt:P20 ?deathPlace. }  # Deathplace
  OPTIONAL { ?artist wdt:P135 ?movement. }  # Artistic movement
  OPTIONAL { ?artist wdt:P106 ?occupation. }  # Occupation
  OPTIONAL { ?artist wdt:P1027 ?paintingSchool. }  # Painting school
  OPTIONAL { ?artist wdt:P737 ?influencedBy. }  # Influenced by
  OPTIONAL { ?artist wdt:P737 ?influencedOn. }  # Influenced on
  OPTIONAL { ?artist wdt:P802 ?pupils. }  # Pupils
  OPTIONAL { ?artist wdt:P108 ?teachers. }  # Teachers
  OPTIONAL { ?artist wdt:P1416 ?friends. }  # Friends and coworkers
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 1***REMOVED***  # Limit results to prevent timeouts
"""

# Initialize SPARQL wrapper
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

try:
    # Fetch response safely
    response = sparql.query().response.read().decode("utf-8")

    # Validate if response is JSON
    if not response.strip().startswith("{"):
        raise ValueError("SPARQL endpoint did not return JSON. Response received:\n" + response)

    # Remove invalid control characters that may cause JSONDecodeError
    cleaned_response = re.sub(r"[\x00-\x1F\x7F]", "", response)

    # Convert JSON response safely
    results = json.loads(cleaned_response)

    # Process results into a DataFrame
    data = []
    for result in results.get("results", {}).get("bindings", []):
        data.append({
            "artist": result["artistLabel"]["value"],
            "nationality": result.get("nationalityLabel", {}).get("value", None),
            "citizenship": result.get("citizenshipLabel", {}).get("value", None),
            "gender": result.get("genderLabel", {}).get("value", None),
            "birth_year": result.get("birthDate", {}).get("value", None),
            "death_year": result.get("deathDate", {}).get("value", None),
            "birth_place": result.get("birthPlaceLabel", {}).get("value", None),
            "death_place": result.get("deathPlaceLabel", {}).get("value", None),
            "movement": result.get("movementLabel", {}).get("value", None),
            "occupation": result.get("occupationLabel", {}).get("value", None),
            "painting_school": result.get("paintingSchoolLabel", {}).get("value", None),
            "influenced_by": result.get("influencedByLabel", {}).get("value", None),
            "influenced_on": result.get("influencedOnLabel", {}).get("value", None),
            "pupils": result.get("pupilsLabel", {}).get("value", None),
            "teachers": result.get("teachersLabel", {}).get("value", None),
            "friends_and_coworkers": result.get("friendsLabel", {}).get("value", None),
        })

    wikidata_df = pd.DataFrame(data)

    # Save the fetched data to a CSV file
    wikidata_df.to_csv("Wikidata_Artists_Info.csv", index=False)

    # Display results
    print(f"Total records fetched: {len(wikidata_df)}")
    print(wikidata_df.head())

except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
except ValueError as ve:
    print(f"ValueError: {ve}")
except Exception as ex:
    print(f"Unexpected error: {ex}")


Total records fetched: 1***REMOVED***
      artist     nationality     citizenship gender            birth_year  \
0  Rembrandt  Dutch Republic  Dutch Republic   male  1606-07-15T00:00:00Z   
1  Rembrandt  Dutch Republic  Dutch Republic   male  1606-07-15T00:00:00Z   
2  Rembrandt  Dutch Republic  Dutch Republic   male  1606-07-15T00:00:00Z   
3  Rembrandt  Dutch Republic  Dutch Republic   male  1606-07-15T00:00:00Z   
4  Rembrandt  Dutch Republic  Dutch Republic   male  1606-07-15T00:00:00Z   

             death_year birth_place death_place                   movement  \
0  1669-10-04T00:00:00Z      Leiden   Amsterdam  Dutch Golden Age painting   
1  1669-10-04T00:00:00Z      Leiden   Amsterdam  Dutch Golden Age painting   
2  1669-10-04T00:00:00Z      Leiden   Amsterdam  Dutch Golden Age painting   
3  1669-10-04T00:00:00Z      Leiden   Amsterdam  Dutch Golden Age painting   
4  1669-10-04T00:00:00Z      Leiden   Amsterdam  Dutch Golden Age painting   

      occupation painting_scho

In [87]:
null_counts = wikidata_df.isnull().sum()

# Print missing values for each column
print("Missing (NaN) values per column:")
print(null_counts)

Missing (NaN) values per column:
artist                       0
nationality                 12
citizenship                 12
gender                       0
birth_year                   0
death_year                 120
birth_place                  0
death_place                125
movement                   953
occupation                   0
painting_school          1***REMOVED***
influenced_by             1678
influenced_on             1678
pupils                    5371
teachers                  7411
friends_and_coworkers    1***REMOVED***
dtype: int64


In [ ]:
#Save CSV for download

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json
import re
from collections import defaultdict

# Define the SPARQL query to fetch as many relevant columns as possible
query = """
SELECT ?artist ?artistLabel ?nationalityLabel ?citizenshipLabel ?genderLabel ?birthDate ?deathDate ?birthPlaceLabel ?deathPlaceLabel 
       ?movementLabel ?occupationLabel ?paintingSchoolLabel ?influencedByLabel ?influencedOnLabel ?pupilsLabel ?teachersLabel ?friendsLabel
WHERE {
  ?artist wdt:P31 wd:Q5;  # Instance of human
         wdt:P106 wd:Q1028181.  # Occupation: artist
  OPTIONAL { ?artist wdt:P27 ?nationality. }  # Country of citizenship
  OPTIONAL { ?artist wdt:P27 ?citizenship. }  # Citizenship
  OPTIONAL { ?artist wdt:P21 ?gender. }  # Gender
  OPTIONAL { ?artist wdt:P569 ?birthDate. }  # Date of birth
  OPTIONAL { ?artist wdt:P570 ?deathDate. }  # Date of death
  OPTIONAL { ?artist wdt:P19 ?birthPlace. }  # Birthplace
  OPTIONAL { ?artist wdt:P20 ?deathPlace. }  # Deathplace
  OPTIONAL { ?artist wdt:P135 ?movement. }  # Artistic movement
  OPTIONAL { ?artist wdt:P106 ?occupation. }  # Occupation
  OPTIONAL { ?artist wdt:P1027 ?paintingSchool. }  # Painting school
  OPTIONAL { ?artist wdt:P737 ?influencedBy. }  # Influenced by
  OPTIONAL { ?artist wdt:P737 ?influencedOn. }  # Influenced on
  OPTIONAL { ?artist wdt:P802 ?pupils. }  # Pupils
  OPTIONAL { ?artist wdt:P108 ?teachers. }  # Teachers
  OPTIONAL { ?artist wdt:P1416 ?friends. }  # Friends and coworkers
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 1***REMOVED***  # Limit results to prevent timeouts
"""

# Initialize SPARQL wrapper
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

try:
    # Fetch response safely
    response = sparql.query().response.read().decode("utf-8")

    # Validate if response is JSON
    if not response.strip().startswith("{"):
        raise ValueError("SPARQL endpoint did not return JSON. Response received:\n" + response)

    # Remove invalid control characters that may cause JSONDecodeError
    cleaned_response = re.sub(r"[\x00-\x1F\x7F]", "", response)

    # Convert JSON response safely
    results = json.loads(cleaned_response)

    # Initialize a dictionary to store unique artist data with lists for multiple values
    artist_data = defaultdict(lambda: {
        "nationality": [], "citizenship": [], "gender": [], "birth_year": None, "death_year": None,
        "birth_place": [], "death_place": [], "movement": [], "occupation": [], "painting_school": [],
        "influenced_by": [], "influenced_on": [], "pupils": [], "teachers": [], "friends_and_coworkers": []
    })

    # Process results into the dictionary, aggregating multiple values into lists
    for result in results.get("results", {}).get("bindings", []):
        artist_name = result["artistLabel"]["value"]

        artist_data[artist_name]["nationality"].append(result.get("nationalityLabel", {}).get("value", None))
        artist_data[artist_name]["citizenship"].append(result.get("citizenshipLabel", {}).get("value", None))
        artist_data[artist_name]["gender"].append(result.get("genderLabel", {}).get("value", None))
        artist_data[artist_name]["birth_place"].append(result.get("birthPlaceLabel", {}).get("value", None))
        artist_data[artist_name]["death_place"].append(result.get("deathPlaceLabel", {}).get("value", None))
        artist_data[artist_name]["movement"].append(result.get("movementLabel", {}).get("value", None))
        artist_data[artist_name]["occupation"].append(result.get("occupationLabel", {}).get("value", None))
        artist_data[artist_name]["painting_school"].append(result.get("paintingSchoolLabel", {}).get("value", None))
        artist_data[artist_name]["influenced_by"].append(result.get("influencedByLabel", {}).get("value", None))
        artist_data[artist_name]["influenced_on"].append(result.get("influencedOnLabel", {}).get("value", None))
        artist_data[artist_name]["pupils"].append(result.get("pupilsLabel", {}).get("value", None))
        artist_data[artist_name]["teachers"].append(result.get("teachersLabel", {}).get("value", None))
        artist_data[artist_name]["friends_and_coworkers"].append(result.get("friendsLabel", {}).get("value", None))

        # Assign unique values for birth and death year (assuming single values)
        if artist_data[artist_name]["birth_year"] is None:
            artist_data[artist_name]["birth_year"] = result.get("birthDate", {}).get("value", None)
        if artist_data[artist_name]["death_year"] is None:
            artist_data[artist_name]["death_year"] = result.get("deathDate", {}).get("value", None)

    # Convert lists with single values into single values
    for artist in artist_data:
        for key in artist_data[artist]:
            if isinstance(artist_data[artist][key], list):
                artist_data[artist][key] = list(filter(None, artist_data[artist][key]))  # Remove None values

    # Convert dictionary to DataFrame
    final_wikidata_df = pd.DataFrame.from_dict(artist_data, orient="index").reset_index().rename(columns={"index": "artist"})

    # Save and display the final DataFrame
    final_wikidata_df.to_csv("Wikidata_Artists_Info_Aggregated.csv", index=False)



except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
except ValueError as ve:
    print(f"ValueError: {ve}")
except Exception as ex:
    print(f"Unexpected error: {ex}")
